# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# List all the databases to confirm if 'uk_food' database was created
databases = mongo.list_database_names()

# Display the list of databases
databases

['admin', 'config', 'local']

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
db = mongo['uk_food']

# List collections in the 'uk_food' database
collections = db.list_collection_names()
print(collections)

[]


In [ ]:
# review the collections in our new database
# List the collections in the uk_food database
collections = db.list_collection_names()

# Display the collections
print(collections)

In [ ]:
# Retrieve and display one document from the establishments collection
document = db.establishments.find_one()

# Pretty print the document
pprint(document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
# Define the new restaurant document
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "",
    "AddressLine3": "Greenwich",
    "AddressLine4": "",
    "PostCode": "SE10 9XYZ",
    "RatingValue": None,  # Not rated yet
    "LocalAuthority": "Greenwich",
    "Halal": True  # New halal restaurant
}


In [ ]:
# Insert the new restaurant into the establishments collection
db.establishments.insert_one(new_restaurant)


In [ ]:
# Check that the new restaurant was inserted
# Confirm the insertion by finding the new restaurant
inserted_restaurant = db.establishments.find_one({"BusinessName": "Penang Flavours"})

# Pretty print the inserted document
pprint(inserted_restaurant)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
result = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"}, 
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  # Only return the BusinessTypeID and BusinessType fields
)

# Pretty print the result
pprint(result)


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant "Penang Flavours" with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)

# Confirm the update by retrieving the updated document
updated_restaurant = db.establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)

In [ ]:
# Retrieve the updated "Penang Flavours" restaurant document
updated_restaurant = db.establishments.find_one({"BusinessName": "Penang Flavours"})

# Pretty print the document to confirm the update
pprint(updated_restaurant)


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
# Count the number of documents where LocalAuthorityName is "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

# Display the result
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})

# Display the number of documents deleted
print(f"Number of documents deleted: {delete_result.deleted_count}")


In [ ]:
# Check if any documents with LocalAuthorityName as "Dover" still exist
dover_exists = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

# Display the result
if dover_exists > 0:
    print(f"There are still {dover_exists} documents with LocalAuthorityName as 'Dover'.")
else:
    print("No documents with LocalAuthorityName as 'Dover' remain.")


In [ ]:
# Check that other documents remain with 'find_one'
# Retrieve and display one remaining document from the establishments collection
remaining_document = db.establishments.find_one()

# Pretty print the document to confirm others remain
pprint(remaining_document)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Find documents where RatingValue is stored as a string and is numeric (1-5)
numeric_ratings = db.establishments.find({
    "RatingValue": {"$in": ["1", "2", "3", "4", "5"]}
})

# Loop through the documents and convert the RatingValue to an integer
for doc in numeric_ratings:
    # Convert RatingValue from string to integer
    rating_value = int(doc['RatingValue'])
    
    # Update the document with the converted integer value
    db.establishments.update_one(
        {"_id": doc["_id"]},
        {"$set": {"RatingValue": rating_value}}
    )

# Confirm the update
print("RatingValue has been updated from string to integer where applicable.")


In [ ]:
# Retrieve one document where longitude and latitude are decimals and RatingValue is an integer
document_check = db.establishments.find_one({
    "geocode.longitude": {"$type": "double"},
    "geocode.latitude": {"$type": "double"},
    "RatingValue": {"$type": "int"}
})
# Check for a document where RatingValue is null
null_rating_check = db.establishments.find_one({"RatingValue": None})

# Pretty print the result
pprint(null_rating_check)
# Pretty print the document to confirm
pprint(document_check)